LOADING THE DATASET

In [ ]:
import pandas as pd
import pyfpgrowth
data=pd.read_csv('..\dataset_file')

DATA PREPROCESSING

In [ ]:
#CONVERTING DATAFRAME TO LIST
lis=data.values.tolist()

#CHANGING LIST TO TRANSCATIONAL DATA 
xy=lis[1][1]
mod_data=[]
for i in lis:
    arr=[]
    for j in i:
        if(id(j)-id(xy))!=0 :
            arr.append(j)
    mod_data.append(arr)

PATTERN GENERATION

In [ ]:
patterns = pyfpgrowth.find_frequent_patterns(mod_data, 0.3)

RULE GENERATION

In [ ]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.8)

CONVERTING RULES INTO DATAFRAME

In [ ]:
data_list=[]
for ante,dente in rules.items():
    arr=[]
    arr.append(ante)
    arr.append(dente[0])
    arr.append(dente[1])
    
    data_list.append(arr)
rules_df=pd.DataFrame(data_list,columns=['Antecedents','Consequents','Confidence'])
rules_df.shape

CALCULATING THE "LIFT" METRIC

In [ ]:
df=pd.DataFrame(mod_data)
df['Items'] = df[df.columns[:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df.drop(df.columns[0:6],axis=1)

def support_count(rhs):  
    count=0
    rhs1=set(str(rhs))
    rhs1.remove("'")
    rhs1.remove('(')
    rhs1.remove(')')
    rhs1.discard(' ')
    for j in df['Items']:
        j=set(j)
        if(rhs1.issubset(j)):
            count=count+1
    return count
rhs_support=[]
for i in rules_df['Consequents']:
    a=support_count(i)
    rhs_support.append(a/len(df))
rules_df['RHS_support']=rhs_support

In [ ]:
rules_df['lift']=rules_df['Confidence']/rules_df['RHS_support']

ACCEPTABLE RULES GENERATED

In [ ]:
rules_df[(rules_df['lift'] >= 1.2) &
(rules_df['Confidence'] >= 0.8) & (rules_df['Confidence'] >=0.3)].sort_values(by='lift')

In [ ]:
DataFrame_Rules.to_csv('filename')